In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.special import zeta

In [7]:
def hoeffding(d, n , delta, emp_var=None):
    delta = delta/d
    return np.sqrt(np.log(2/delta)/(2*n))

def h_foo(v, delta):
    v = max(v,2)
    return 2*np.log(np.pi*np.log(v)/np.sqrt(6)) + np.log(2/delta)


def seq_hoeffding(d, n , delta, emp_var=None):
    delta = delta/d
    return np.sqrt( 1.064*(n/2)*h_foo(n/2,delta))/n

def seq_bernstein(d, n , delta, emp_var=None):
    delta = delta/d
    emp_var = max(np.max(emp_var),1)
    return (np.sqrt( 1.064*emp_var*h_foo(emp_var,delta) + 1.76*h_foo(emp_var,delta)**2) + 1.33*h_foo(emp_var,delta))/n

def seq_bernstein2(d, n , delta, emp_var=None):
    delta = delta/d
    emp_var = max(np.max(emp_var[0]),1)
    return (np.sqrt( 1.064*emp_var*h_foo(emp_var,delta) + 1.76*h_foo(emp_var,delta)**2) + 1.33*h_foo(emp_var,delta))/n

def seq_u_bernstein2(d, n , delta, emp_var=None):
    delta = delta/d
    emp_var = max(np.max(emp_var[0]),1)
    return np.sqrt(d)*(np.sqrt( 1.064*emp_var*h_foo(emp_var,delta) + 1.76*h_foo(emp_var,delta)**2) + 1.33*h_foo(emp_var,delta))/n

def vec_seq_bernstein(d, n , delta, emp_var=None):
    emp_var = max(np.max(emp_var[1]),1)
    #print(np.sqrt(emp_var)/n)
    return (np.sqrt( 1.064*emp_var*h_foo(emp_var,delta) + 1.76*16*h_foo(emp_var,delta)**2) + 1.33*4*h_foo(emp_var,delta))/n

def emp_bernstein(d, n , delta, emp_var):
    delta = delta/d
    emp_var = np.max(emp_var[1])
    return emp_var*np.sqrt(2*np.log(3/delta)/n) + 3*np.log(3/delta)/n


def vector_bound(d, n , delta):
    return np.sqrt((4*d+np.log(1/delta))/n)

def zeta_vector_bound(d, n , delta, emp_var=None):
    scale = n**2/(np.pi**2/6)
    return np.sqrt((4*d+np.log(1/(delta/scale)))/n)


def zeta_direct_bound(d, n , delta, emp_var=None):
    scale = n**2/(np.pi**2/6)
    delta = delta/scale
    return d/np.sqrt(n) * (2*np.sqrt(d) + np.sqrt(np.log(1/delta)))


def empirical_bernstein(n , delta, emp_var):
    return emp_var*np.sqrt(2*np.log(3/delta)/n) + 3*np.log(3/delta)/n


def vector_u_hoeffding(d, n , delta, emp_var=None):
    return np.sqrt(d)*hoeffding(d, n , delta, emp_var)

def vector_u_bernstein(d, n , delta, emp_var=None):
    delta = delta/d
    return np.linalg.norm(emp_var*np.sqrt(2*np.log(3/delta)/n) + 3*np.log(3/delta)/n)
    
def zeta_vector_u_bernstein(d, n , delta, emp_var=None):
    scale = n**2/(np.pi**2/6)
    delta = (delta/scale)/d
    return np.linalg.norm(emp_var*np.sqrt(2*np.log(3/delta)/n) + 3*np.log(3/delta)/n)

def zeta_vector_u_hoeffding(d, n , delta, emp_var=None):
    scale = n**2/(np.pi**2/6)
    delta = delta/scale
    return np.sqrt(d)*hoeffding(d, n , delta, emp_var)

def exp_uniform_bound(name, bound_foo, d, n , delta, emp_var):
    scale = 2**n
    return bound_foo(d, n , delta/scale, emp_var)

def point_bound(name, bound_foo,  d, n , delta, emp_var):
    return bound_foo(d, n , delta, emp_var)

def zeta_uniform_bound(name, bound_foo,  d, n , delta, emp_var):
    scale = n**2/(np.pi**2/6)
    return bound_foo(d, n , delta/scale, emp_var)


def confidence_sequence(name, bound_foo,  d, n , delta, emp_var):
    if "b" in name:
        return seq_bernstein(d, n , delta, emp_var)
    else:
        return seq_hoeffding(d, n , delta, emp_var)
        
    
def estimate_dirichlet_probabilities(alpha, n):
    p = np.random.dirichlet(alpha)
    samples = np.random.choice(len(p), size=n, p=p)
    counts = np.bincount(samples, minlength=len(p))
    empirical = counts / n
    variance = empirical * (1 - empirical) / n
    return p, empirical, variance
    

def run_point_experiment(p, n, delta=0.05):
    d = len(p)
    samples = np.random.choice(d, size=n, p=p)
    bound_foos = [("h",hoeffding),("b", emp_bernstein),("v",vector_bound)]
    data = []
    
    for i in range(n):
        counts = np.bincount(samples[:i], minlength=len(p))
        empirical = counts / n
        variance = empirical * (1 - empirical) / n
        for c, f in bound_foos:
            data.append([i,c, f(d,i+1,delta, variance )])

    return data



def run_unif_experiment(p, n, delta=0.05):
    d = len(p)
    samples = np.random.choice(d, size=n, p=p)
    one_hot = np.eye(d)[samples]
    bound_foos = [("Hoeffding",hoeffding)]
    unf_foos = [("$polynomial$",zeta_uniform_bound),("$exponential$",exp_uniform_bound),  ("$ours$", confidence_sequence)]
    data = []
    var_sum = 0
    for i in range(1,n):
        mean = one_hot[:i].mean(axis=0)
        vari = one_hot[:i].var(axis=0)
        var_sum += (one_hot[i]-mean)**2
        for c, f in bound_foos:
            for u, uf in unf_foos:
                data.append([i,u, uf(c,f, d,i+1,delta*d, (var_sum,vari) )])
    df = pd.DataFrame(data, columns=["$Time$", "$Bound$", "$Error$"])
    return df

def get_distr(d, distr):
    if "unif" in distr:
        return [1/d]*d
    if "expo" in distr:
        p = [np.exp(-i) for i in range(d)]
        return p / np.sum(p)
    if "poly" in distr:
        p = [1 / (i + 1) for i in range(d)]
        return p / np.sum(p)
        
def run_vec_unif_experiment(n, distr, delta=0.05):
    data = []
    bound_foos = [("l2_seq_vec_bern", vec_seq_bernstein),
                  ("l2_zeta_uni_hoff",zeta_vector_u_hoeffding),
                   ("l2_seq_uni_bern",seq_u_bernstein2),
                  ("l2_zeta_vec_hoeff",zeta_vector_bound), ("linf_seq_uni_bern",seq_bernstein2),
                 ("direct_zeta_vec_hoeff",zeta_direct_bound)]
   
    for d in [2,5,10,100,1000]:
        p = get_distr(d, distr)
        samples = np.random.choice(d, size=n, p=p)
        one_hot = np.eye(d)[samples]
        var_sum = 0
        vvar_sum = 0
        for i in range(1,n):
            mean = one_hot[:i].mean(axis=0)
            vari = one_hot[:i].var(axis=0)
            var_sum += (one_hot[i]-mean)**2
            vvar_sum += (np.linalg.norm(one_hot[i]-mean)**2)
            for c, f in bound_foos:
                data.append([d, i, c, f(d,i+1,delta, (var_sum,vvar_sum) )])
    df = pd.DataFrame(data, columns=["dimension","time", "type", "value"])
    df["experiment"] = distr
    return df


def run_vec_unif_experiment2(n, distr, delta=0.05):
    data = []
    bound_foos = [("l2_seq_vec_bern", vec_seq_bernstein),
                  ("l2_zeta_uni_hoff",zeta_vector_u_hoeffding),
                   ("l2_seq_uni_bern",seq_u_bernstein2),
                  ("l2_zeta_vec_hoeff",zeta_vector_bound), ("linf_seq_uni_bern",seq_u_bernstein2)]
   
    for d in [2,5,10,100,1000]:
        p = get_distr(d, distr)
        samples = np.random.choice(d, size=n, p=p)
        one_hot = np.eye(d)[samples]
        var_sum = 0
        vvar_sum = 0
        for i in range(1,n):
            mean = one_hot[:i].mean(axis=0)
            vari = one_hot[:i].var(axis=0)
            var_sum += (one_hot[i]-mean)**2
            vvar_sum += (np.linalg.norm(one_hot[i]-mean)**2)
            for c, f in bound_foos:
                data.append([d, i, c, f(d,i+1,delta, (var_sum,vvar_sum) )/np.sqrt(d)])
    return data


def run_vec_unif_experiment3(n, distr, delta=0.05):
    data = []
    bound_foos = [("l2_seq_vec_bern", vec_seq_bernstein),
                  ("l2_zeta_uni_hoff",zeta_vector_u_hoeffding),
                   ("l2_seq_uni_bern",seq_u_bernstein2),
                  ("l2_zeta_vec_hoeff",zeta_vector_bound), ("linf_seq_uni_bern",seq_u_bernstein2)]
   
    for d in [2,5,10,100,1000]:
        p = get_distr(d, distr)
        samples = np.random.choice(d, size=n, p=p)
        one_hot = np.eye(d)[samples]
        var_sum = 0
        vvar_sum = 0
        for i in range(1,n):
            mean = one_hot[:i].mean(axis=0)
            vari = one_hot[:i].var(axis=0)
            var_sum += (one_hot[i]-mean)**2
            vvar_sum += (np.linalg.norm(one_hot[i]-mean)**2)
            for c, f in bound_foos:
                data.append([d, i, c, (f(d,i+1,delta, (var_sum,vvar_sum) )/np.sqrt(d))**2])
    return data

def run_vec_unif_experiment4(n, distr, delta=0.05):
    data_dfs = []
    for name,expe in [("norm",run_vec_unif_experiment),("root_mse",run_vec_unif_experiment2),("mse",run_vec_unif_experiment3)]:
        print(name)
        data = expe(n,distr)
        df = pd.DataFrame(data, columns=["dimension","time", "type", "value"])
        df["experiment"] = name
        data_dfs.append(df)
    return pd.concat(data_dfs)
            
        



In [8]:
df = run_unif_experiment([1/2, 1/2], 1000)

In [9]:
df

$Time$        $Bound$   $Error$
0          1   $polynomial$  1.069752
1          1  $exponential$  1.126407
2          1         $ours$  0.958460
3          2   $polynomial$  0.947664
4          2  $exponential$  0.980503
...      ...            ...       ...
2992     998  $exponential$  0.590271
2993     998         $ours$  0.064615
2994     999   $polynomial$  0.092214
2995     999  $exponential$  0.590269
2996     999         $ours$  0.064584

[2997 rows x 3 columns]

In [14]:
plt.rcParams.update({'font.size': 14})
sns.relplot(kind="line", data=df, x="$Time$", y="$Error$", hue="$Bound$")
plt.yscale("log")
plt.savefig("cav_comp.pdf")

In [1]:
n = 10000
df = run_vec_unif_experiment(n, "unif")
sns.relplot(kind="line", data=df, x="time", y="value", hue="type", col="dimension", row="experiment")
plt.yscale("log")
plt.savefig("direct_unif_vec_comp.pdf")

NameError: name 'run_vec_unif_experiment' is not defined

In [ ]:
n = 10000
df = run_vec_unif_experiment4(n, "poly")
sns.relplot(kind="line", data=df, x="time", y="value", hue="type", col="dimension", row="experiment")
plt.yscale("log")
plt.savefig("poly_vec_comp.pdf")

In [ ]:
sns.relplot(kind="line", data=df[df["dimension"]==5], x="time", y="value", hue="type", col="dimension", row="experiment")


In [ ]:
samples = np.random.choice(2, size=10, p=[1/2,1/2])
    
one_hot.mean(axis=0)
cumulative_mean = np.cumsum(one_hot, axis=0) / np.arange(1, samples.shape[0]+1)[:, None]
emp_var = np.sum((one_hot- cumulative_mean)**2, axis=0)

In [ ]:
sns.relplot(kind="line", data=df, x="t", y="v", hue="b", style="u")
plt.yscale("log")


In [ ]:
((np.e*(1/4)+np.e**(-1/4))/np.sqrt(2))**2

In [ ]:
((np.sqrt(np.e)+1)/2)

In [ ]:
#sns.relplot(kind="line", data=df, x="t", y="v", hue="b")
#plt.yscale("log")

In [ ]:
f = lambda x,y : x+y
g = lambda x: f(*x)

In [ ]:
g([1,2])

In [ ]:
SeqBern       8530864
SeqHoef      39541552
Clopper-sq  106500830
Hoeff-sq    112451255
Clopper-exp 115679470
Hoeff-exp   303197120

In [ ]:
13.5*8530864

In [ ]:
d = 1000
p = [1/d]*d
n = 10000
data=run_vec_unif_experiment(p,n)
df = pd.DataFrame(data, columns=["time","unif", "type", "value"])
sns.relplot(kind="line", data=df, x="time", y="value", hue="type", style="unif")
plt.yscale("log")
plt.savefig("1000_vector.pdf")

In [ ]:
v = max(1, sample count oder variance)
l(v) = log(h(log(v)) + log(2 / delta)
k1 = (e^.25 + e^-.25) / sqrt(2)
k2 = (sqrt(e) + 1) / 2

S_alpha = sqrt(k1^2 * v * l(v) + k2^2 * c^2 * l(v)^2) + k2 * c * l(v)
offset = S_alpha / sample count
return [emprical p - offset, empirical p + offset]

In [ ]:
double v = Math.max(m, sampleCount);
double alpha = confidence / 2;
double lv = Math.log(h.applyAsDouble(Math.log(v / m) / Math.log(eta))) + Math.log(1.0 / alpha);
double k1 = (Math.pow(eta, .25) + Math.pow(eta, -.25)) / Math.sqrt(2.0);
double k2 = (Math.sqrt(eta) + 1) / 2;
double Salpha = Math.sqrt(k1 * k1 * v * lv + k2 * k2 * c * c * lv * lv) + k2 * c * lv;

In [ ]:
1509/50

In [ ]:
 2032838 / 10211972